# Download and Extract Data

In [ ]:
 !git clone "https://github.com/Text-Mining/Useful-Corpora-for-Text-Mining-in-Persian-Language"

Cloning into 'Useful-Corpora-for-Text-Mining-in-Persian-Language'...
remote: Enumerating objects: 18, done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (18/18), 478.78 MiB | 10.60 MiB/s, done.
Updating files: 100% (8/8), done.


In [ ]:
!pip install unrar
!unrar x '/content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part01.rar'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part01.rar

Extracting  farsnews.json                                                  0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38%

Extracting from /content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part02.rar

...         farsnews.json                                                 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55%

#Read Data

In [ ]:
import json
counter = 0
N = 1000
news=[] # { CategoryPanel[], NewsDate, NewsTitle, NewsSummary, NewsBody, GetComments: { CommentsJsonArray[] } }

# Read just one line of data for now
for line in open('farsnews.json','r', encoding='utf-8-sig'):
  news.append(json.loads(line))
  counter+=1
  if(counter == N):
    break

#Clean Data

In [ ]:
for i in range(len(news)):
  if i>len(news):
    break
  if i<=len(news) and list(news[i].keys()).count('NewsBody')==0:
    del news[i]
  if str(type(news[i]['NewsBody']))=="<class 'list'>":
    news[i]['NewsBody'] =news[i]['NewsBody'][0]

# Modules Installation

In [ ]:
# hazm installation
!pip install hazm
# Docs: www.roshan-ai.ir/hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=0a5e25c04d724860cb68c5a3f4b67239580dfd78f281aa83d61a4069a4aaf5b7
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180368 sha256=e40a146ca78db26820c5924b280d05ce2f5b251f09f75503c21d10682b4e5341
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

# Process Data With Hazm

In [ ]:
from hazm import *

In [ ]:
def hazm_normalize(text):
  # preprocesing => remove '\n'
  removeNewline = text.replace("\r\n", "")

  normalizer = Normalizer()
  normalized = normalizer.normalize(removeNewline)
  normalized = normalizer.character_refinement(normalized)
  normalized = normalizer.punctuation_spacing(normalized)

  # print("(HAZM): normalized body: ", normalized)
  return normalized

In [ ]:
def hazm_tokenize(text):
  # to split text to words
  tokenized_words = word_tokenize(text)
  tokenized_text = sent_tokenize(text)

  # print("(HAZM): tokenized body: ", tokenized_words)
  return tokenized_words, tokenized_text

In [ ]:
def hazm_stem(words):
  stemmer = Stemmer()
  stemmed = []
  for word in words:
    stemmed.append(stemmer.stem(word))

  # print("(HAZM): stemmer result: ", stemmed)
  return stemmed

In [ ]:
def hazm_process_news(news_body):
  # news_body = news['NewsBody']
  normalized_body = hazm_normalize(news_body)
  tokenized_body = hazm_tokenize(normalized_body)
  stemmed_body = hazm_stem(tokenized_body[0])

  return stemmed_body


#Making Posting-List

In [ ]:
data = {}
for i in range(len(news)):
  docId = i + 1
  tokens = hazm_process_news(news[i]['NewsBody'])
  tokens_freq = {}

  for token in tokens:
    if len(token) >= 3:
      if token in tokens_freq:
        tokens_freq[token] += 1
      else:
        tokens_freq[token] = 1

  for token in tokens_freq:
    if token in data:
      data[token]['freq'] += tokens_freq[token]
      data[token]['docs'].append({
          'id': docId,
          'freq': tokens_freq[token]
          })
    else:
      data[token] = {
          'freq': tokens_freq[token],
          'docs': [
              {
                  'id': docId,
                  'freq': tokens_freq[token]
               }
              ]
            }

print(len(data))

17352


#Mapper Function
returns a list of tokens with their documents' IDs

In [ ]:
def mapper_function(token_index):
  token = data_keys[token_index] # محمد
  token_data = data_vals[token_index] # {freq: F, docs: [{ id: ID, freq: DOC_FREQ }, ...]}
  docs_list = [docs for docs in token_data['docs']]
  values[token] = docs_list

#List of all tokens with their docID

In [ ]:
data_keys = [*data]
data_vals = [*data.values()]


values = {}
# {token : docs[] }

for index, _ in enumerate(data_keys):
  mapper_function(index)

print(values)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#User Input

In [ ]:
query = input()
terms = query.split(".")
terms.sort(key=lambda x:x.startswith("!"))
docs = []
for term in terms:
  if term not in values:
    continue
  docIds = [id for id in values[term]['id']]
  if term.startsWith("!"):
    docs = [id for id in [*docs, docIds] if id in docs and id not in docIds]
  else:
    docs = [id for id in [*docs, *docIds] if id in docs and id in docIds]

#Creating Model with Datafram

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
col = np.arange(len(news)+1)
df = pd.DataFrame(0,index=col, columns=data_keys)
for token in values:
  for d in values[token]:
    df[token][d['id'] - 1] = d['freq']
df

,محمد,نور,گفتوگوبا,خبرنگار,ورزش,خبرگزار,فارس,مورد,پیروز,ذوب‌آهن,...,هنرپیشه,نتوانسته_بود,چندده‌میلیاردی,۱۲:۳۰,سعادتمند,میرشاد,پندار,خمارلو,موکد,راهرو
0,1,2,1,1,1,1,1,2,1,10,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,7,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
995,0,0,0,1,0,1,1,0,0,0,...,1,1,1,1,0,0,0,0,0,0
996,0,0,0,1,1,1,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0
997,0,0,0,0,0,1,1,3,0,0,...,0,0,0,0,0,0,1,1,1,1


##The Input Queries

In [ ]:
all_queries = [
   'آخرین مهلت ثبتنام آزمون کارشناسی ارشد',
 'واریز سود سهام عدالت کارگران و زنان سرپرست',
 'تبریک مدیرعامل بانک ملت به مناسبت روز مهندسی',
 'به خیابان آمدن هزاران رومانیایی در اعتراض به حکم جدید دولت',
 'طرح استیضاح رئیس جمهور',
 'واکسیناسیون فلج اطفال در سیستان وبلوچستان',
 'رقابتهای اسکی صحرانوردی قهرمان جهانی',
 'روش های پیشگیری و مهمترین اصل در تشخیص بههنگام سرطان',
 'ترکیب تیم منتخب ایران برای حضور در جام جهانی کشتی آزاد',
 'فروش و توزیع میوههای لاکچری قاچاق',
 'معاونت پژوهشی موسسه حکمت و فلسفه ایران',
 'تساوی بدون گل لیورپول و بایرن مونیخ در دیدار رفت یک هشت نهایی لیگ قهرمانان اروپا'
]

##Jccard

In [ ]:
def create_query_df(tokens):
  df = pd.DataFrame()
  for t in tokens:
    if t in df.columns:
      df[t][0] += 1
    else:
      df[t] = [1]
  return df

In [ ]:
def create_query_df1(query):
  tokens = hazm_process_news(all_queries[0])
  df = pd.DataFrame()
  for t in tokens:
    if t in df.columns:
      df[t][0] += 1
    else:
      df[t] = [1]
  return df


In [ ]:
def jaccard(query):
  query_df = create_query_df(hazm_process_news(query))
  common_tokens =  [x for x in query_df.columns if x in df.columns]
  all_scores = {}
  for i in range(df.index.size):
    intersection = len([x for x in common_tokens if df.iloc[i][x]>=1])
    union = sum(df.iloc[i]) + len(query_df) - len(common_tokens)
    if union > 0:
      all_scores[i] = intersection/math.sqrt(union)
  return(all_scores)

In [ ]:
def print_jccard_top_10(query):
  j_score = jaccard(query)
  jccard_score = sorted(j_score.items(), key=lambda x:x[1], reverse=True)
  print(jccard_score[:10])

##Normalized tf

In [ ]:
def normalized_tf(query):
  query_df = create_query_df(hazm_process_news(query))
  tf_list = {}
  for i in range(df.index.size):
    TOKENS_COUNT = sum(df.iloc[i])
    tf_list[i] = 0
    if TOKENS_COUNT > 0:
      for term in query_df:
        tf_i = 0
        if term in df.columns:
          tf_i = df.iloc[i][term]/TOKENS_COUNT
      tf_list[i] += tf_i
  return tf_list

In [ ]:
def print_normalized_tf_top_10(query):
  tf = normalized_tf(query)
  tf_sorted = sorted(tf.items(), key=lambda x:x[1], reverse=True)
  print(tf_sorted[:10])

#tf-idf

generate document-frequency based dataframe

In [ ]:
DF_ROW = 0
IDF_ROW = 1
col = np.arange(2)
idf_dataframe = pd.DataFrame(0,index=col, columns=data_keys)

# calculating document frequency
for token in values:
  idf_dataframe[token][DF_ROW] = len(values[token])

# calculating inverse document frequency
TOTAL_DOCUMENTS_COUNT = len(news)
for token in idf_dataframe.columns:
  document_frequency = idf_dataframe[token][DF_ROW]
  idf_dataframe[token][IDF_ROW] = math.log10(TOTAL_DOCUMENTS_COUNT/document_frequency)

idf_dataframe

,محمد,نور,گفتوگوبا,خبرنگار,ورزش,خبرگزار,فارس,مورد,پیروز,ذوب‌آهن,...,هنرپیشه,نتوانسته_بود,چندده‌میلیاردی,۱۲:۳۰,سعادتمند,میرشاد,پندار,خمارلو,موکد,راهرو
0,127.000000,24.000000,25.000000,408.00000,82.000000,851.000000,942.00000,244.000000,85.000000,14.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.895327,1.618919,1.601191,0.38847,1.085317,0.069201,0.02508,0.611741,1.069712,1.853003,...,2.999131,2.999131,2.999131,2.999131,2.999131,2.999131,2.999131,2.999131,2.999131,2.999131


In [ ]:
def tf_idf(query):
  query_df = create_query_df(hazm_process_news(query))
  tf_idf_list = []
  for i in range(TOTAL_DOCUMENTS_COUNT):
    tf_idf_val = 0
    for word in query_df.columns:
      if word in df.columns:
        freq = df[word][i]
        tf = 1 + math.log10(freq) if freq > 0 else 0
        idf = idf_dataframe[word][IDF_ROW]
        tf_idf_val = tf*idf
      # else:
      #   freq = 0
      #   tf = 0
      #   idf = 0
      #   tf_idf_val = tf*idf
    tf_idf_list.append((i, tf_idf_val))
  return tf_idf_list

In [ ]:
def print_tf_idf_top_10(query):
  result = tf_idf(query)
  sorted_result = sorted(result, key=lambda x:x[1], reverse=True)
  print(sorted_result[:10])

#Output Result

In [ ]:
for index, query in enumerate(all_queries):
  print(index+1)
  print_jccard_top_10(query)
  print_normalized_tf_top_10(query)
  print_tf_idf_top_10(query)
  print("---------")

1
[(19, 0.2727272727272727), (865, 0.21566554640687682), (921, 0.19802950859533489), (157, 0.1889822365046136), (18, 0.173421993904824), (905, 0.1708715315433522), (225, 0.16169041669088866), (539, 0.15961737689352443), (425, 0.15617376188860607), (907, 0.15617376188860607)]
[(905, 0.02127659574468085), (18, 0.014598540145985401), (820, 0.01263537906137184), (181, 0.011627906976744186), (230, 0.0111731843575419), (865, 0.011111111111111112), (921, 0.009433962264150943), (737, 0.008771929824561403), (19, 0.008), (52, 0.00641025641025641)]
[(820, 2.835422203832089), (48, 2.269940302658434), (905, 2.269940302658434), (18, 1.9993351342613286), (150, 1.9993351342613286), (173, 1.9993351342613286), (230, 1.9993351342613286), (325, 1.9993351342613286), (442, 1.9993351342613286), (737, 1.9993351342613286)]
---------
2
[(846, 0.3952847075210474), (450, 0.18569533817705186), (770, 0.18257418583505536), (518, 0.16275769175423188), (36, 0.15075567228888181), (685, 0.15032920560056579), (625, 0.149